In [1]:
#!/usr/bin/env python3

from Bio.SeqUtils import MeltingTemp as mt

from Bio.SeqUtils import IsoelectricPoint as isp
from Bio.Seq import Seq, MutableSeq
from Bio.Alphabet import generic_rna
from Bio import SeqIO

from Bio import Alphabet
from Bio.Data import IUPACData

from nupack_my import * # nupack_wrapper1 package ahtung
from distances import *
from characteristics import *
from additional_tools import *

import os
import re
import math

import csv

/home/marina/anaconda3/lib/python3.5/site-packages/Bio/__init__.py:128: BiopythonWarning: You may be importing Biopython from inside the source tree. This is bad practice and might lead to downstream issues. In particular, you might encounter ImportErrors due to missing compiled C extensions. We recommend that you try running your code from outside the source tree. If you are outside the source tree then you have a setup.py file in an unexpected directory: /home/marina/anaconda3/lib/python3.5/site-packages.
  format(_parent_dir), BiopythonWarning)


In [12]:
# https://en.wikipedia.org/wiki/Damerau%E2%80%93Levenshtein_distance
# https://en.wikipedia.org/wiki/Levenshtein_distance
# https://en.wikipedia.org/wiki/Mahalanobis_distance

In [94]:
#list_name = "RF00050.fa RF00059.fa RF00080.fa RF00162.fa RF00167.fa"
#list_name = "RF00168.fa RF00174.fa RF00234.fa RF00379.fa RF00380.fa"
#list_name = "RF00442.fa RF00504.fa RF00521.fa RF00522.fa RF00634.fa"
#list_name = "RF01054.fa RF01055.fa RF01056.fa RF01057.fa RF01482.fa"
#list_name = "RF01510.fa RF01689.fa RF01725.fa RF01727.fa RF01734.fa"
#list_name = "RF01739.fa RF01750.fa RF01767.fa RF01786.fa RF01826.fa"
#list_name = "RF01831.fa RF02680.fa RF02683.fa RF02885.fa RF02912.fa"
#list_name = "RF03057.fa RF03058.fa RF03071.fa RF03072.fa"

#list_name = list_name.split()

list_name = ['RF00050.fa', 'RF00059.fa', 'RF00162.fa', 'RF00167.fa',
                    'RF00168.fa', 'RF00504.fa', 'RF01725.fa', 'RF01727.fa',
                    'RF01739.fa']

err_file = open("error.txt", "a+")
err_file.write("GROUP ID" + '\t' + "RNA ID" + '\t' + "SEQUENCE")


save_path1 = os.getcwd()+'/'

for fasta_name in list_name:
        
    save_path = save_path1 + fasta_name.split(".")[0]
    
    if not(os.path.exists(save_path)):
        os.mkdir(save_path)
        
    file_output_name1 = fasta_name.split(".")[0] + ".txt"


    vienna_source_file_name = "res_" + file_output_name1
    vienna_source_file = open(vienna_source_file_name, "r")
    vienna_lines = vienna_source_file.readlines()

    csv_file_name1 = fasta_name.split(".")[0] +".csv"
    csv_file_name = os.path.join(save_path, csv_file_name1)
    
    fieldnames = ["RNA ID","LENGTH","G-С PERCENTAGE", "WALLACE TEMP","GC TEMP","NN TEMP","SIMPLE MASS",
                  "MOLECULAR MASS","NUPACK MFE", "VIENNA MFE", "NUPACK DOT-BRACKET",
                  "VIENNA DOT-BRACKET","Hammington distance(NUPACK, VIENNA)",
                  "Damerau-Levenshtein distance(NUPACK, VIENNA)"]
    
    csv_file = open(csv_file_name, "w+", newline='\n')
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(fieldnames)
    
    for seq_record in SeqIO.parse(fasta_name , "fasta"):
        
        dic = seq_record.seq
        i = seq_record.id
        
        new_RNA = [str(dic).encode('utf-8')]

        G, C, A, L = nucl_cont(dic)
        Tw, Tgc, Tnn = temperatures(dic)
        mw, mass = weights(dic,G,C,A,L)

        new_RNA = [str(dic).encode('utf-8')]
        MFE = mfe(new_RNA, material='rna')

        #---------------------------------------------------------------------------------
        for j in range(2,len(vienna_lines),3):
            vienna_id = vienna_lines[j-2].split(' ')[0].replace('>', '')

            if vienna_id == i:
                vienna_dot_bracket = vienna_lines[j].split(' ')[0]
                fixed_vienna_line = re.sub(r'\( ','(', vienna_lines[j])
                vienna_mfe = fixed_vienna_line.split(' ')[1].replace('(','').replace(')','').replace('\\n','')
                break    
        #---------------------------------------------------------------------------------
        
        if (len(MFE) == 0):
            err_file.write(fasta_name.split(".")[0] + '\t' + i + '\t' + str(dic) + '\n')
            MFE = [["-"*len(dic), "0."]]
        
        values = [i,L,round(((G + C) / L) * 100, 2),Tw,Tgc,Tnn,mass, mw,float(MFE[0][1]),float(vienna_mfe.split('\n')[0]),MFE[0][0],vienna_dot_bracket,hamming_distance(MFE[0][0], vienna_dot_bracket) ,damerau_levenshtein_distance(MFE[0][0], vienna_dot_bracket)]
        

        
        writer.writerow(values)
        
        
    csv_file.close()
    vienna_source_file.close()


    
    
err_file.close()

In [15]:
#OLD VERSION

In [ ]:
#list_name = "RF00050.fa RF00059.fa RF00080.fa RF00162.fa RF00167.fa"
#list_name = "RF00168.fa RF00174.fa RF00234.fa RF00379.fa RF00380.fa"
#list_name = "RF00442.fa RF00504.fa RF00521.fa RF00522.fa RF00634.fa"
#list_name = "RF01054.fa RF01055.fa RF01056.fa RF01057.fa RF01482.fa"
#list_name = "RF01510.fa RF01689.fa RF01725.fa RF01727.fa RF01734.fa"
#list_name = "RF01739.fa RF01750.fa RF01767.fa RF01786.fa RF01826.fa"
#list_name = "RF01831.fa RF02680.fa RF02683.fa RF02885.fa RF02912.fa"
#list_name = "RF03057.fa RF03058.fa RF03071.fa RF03072.fa"
list_name = ['RF00050.fa']
#list_name = list_name.split()
err_file = open("error.txt", "a+")
err_file.write("GROUP ID" + '\t' + "RNA ID" + '\t' + "SEQUENCE")


save_path1 = os.getcwd()+'/'

for fasta_name in list_name:
        
    save_path = save_path1 + fasta_name.split(".")[0]
    
    if not(os.path.exists(save_path)):
        os.mkdir(save_path)
        
    file_output_name1 = fasta_name.split(".")[0] + ".txt"
    file_output_name = os.path.join(save_path, file_output_name1)

    file_out = open(file_output_name, "a+")
    file_out.write("RNA ID" + '\t' + "SEQUENCE" + '\t' + "NUPACK DOT-BRACKET" + '\t' + "NUPACK MFE" + '\t' + "VIENNA DOT-BRACKET" + '\t' + "VIENNA MFE" + '\t'+ 'Hammington distance(NUPACK, VIENNA)'+ '\t'+'Damerau-Levenshtein distance(NUPACK, VIENNA)' +'\n')

    table_name1 = fasta_name.split(".")[0] + "_table.txt"
    table_name = os.path.join(save_path, table_name1)

    table_file = open(table_name, "a+")
    table_file.write("RNA ID" + '\t' + "LENGTH" + '\t' + "G-С PERCENTAGE" + '\t' + "WALLACE TEMP" + '\t' + "GC TEMP" + '\t' + "NN TEMP" + '\t' + "SIMPLE MASS" + '\t' + "MOLECULAR MASS" + '\t' + "NUPACK MFE" + '\t' + "VIENNA MFE" + '\n');


    vienna_source_file_name = "res_" + file_output_name1
    vienna_source_file = open(vienna_source_file_name, "r")
    vienna_lines = vienna_source_file.readlines()
        
    for seq_record in SeqIO.parse(fasta_name , "fasta"):
        
        
        dic = seq_record.seq
        i = seq_record.id


        new_RNA = [str(dic).encode('utf-8')]

        G, C, A, L = nucl_cont(dic)
        Tw, Tgc, Tnn = temperatures(dic)
        mw, mass = weights(dic,G,C,A,L)

        new_RNA = [str(dic).encode('utf-8')]
        MFE = mfe(new_RNA, material='rna')

        #---------------------------------------------------------------------------------
        for j in range(2,len(vienna_lines),3):


            vienna_id = vienna_lines[j-2].split(' ')[0].replace('>', '')

            if vienna_id == i:
                vienna_dot_bracket = vienna_lines[j].split(' ')[0]
                fixed_vienna_line = re.sub(r'\( ','(', vienna_lines[j])
                vienna_mfe = fixed_vienna_line.split(' ')[1].replace('(','').replace(')','').replace('\\n','')
                break    
        #---------------------------------------------------------------------------------


        if (len(MFE) == 0):
            err_file.write(fasta_name.split(".")[0] + '\t' + i + '\t' + str(dic) + '\n')
            MFE = [["-"*len(dic), "0."]]



        file_out.write(i + '\t' + str(dic) + '\t' + str(MFE[0][0]) + '\t' + str(MFE[0][1]) + '\t' + vienna_dot_bracket + '\t' + str(vienna_mfe) + '\t'+ str(hamming_distance(MFE[0][0], vienna_dot_bracket))+'\t'+str(damerau_levenshtein_distance(MFE[0][0], vienna_dot_bracket)) +'\n')
        table_file.write(i + '\t' + str(L) + '\t' + str(round(((G + C) / L) * 100, 2)) + '\t' + str(Tw) + '\t' + str(Tgc) + '\t' + str(Tnn) + '\t' + str(mass) + '\t' + str(mw) + '\t' + str(MFE[0][1]) + '\t' + str(vienna_mfe) + '\n')

    file_out.close()
    vienna_source_file.close()
    table_file.close()


    
    
err_file.close()